# Consumindo API (GET)

In [ ]:

# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests_with_caching
import json
# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_movies_from_tastedive("Bridesmaids")
def get_movies_from_tastedive(name, tp="movies", limit=5):
    urlbase = "https://tastedive.com/api/similar"
    params = dict()
    params["q"] = name 
    params["type"] = tp
    params["limit"] = 5
    resp = requests_with_caching.get(urlbase, params=params)
    date = json.loads(resp.text)
    return date

def extract_movie_titles(date):
    return [name["Name"] for name in date["Similar"]["Results"]]

def get_related_titles(args):
    if len(args) > 0:
        dates = list((map(extract_movie_titles, (map(get_movies_from_tastedive, args)))))
        return list(set(dates[0]+dates[1]))
    else: 
        return []

def get_movie_data(name, tp="json"):
    urlbase = "http://www.omdbapi.com/"
    params = dict()
    params["t"] = name 
    params["r"] = tp
    resp = requests_with_caching.get(urlbase, params=params)
    date = json.loads(resp.text)
    return date

def get_movie_rating(movieInfo):
    if len(movieInfo['Ratings']) > 1:
        num = movieInfo['Ratings'][1]["Value"]
        return int(num.replace("%", ""))
    else:
        return 0
    

def get_sorted_recommendations(args):
    movies = get_related_titles(args)
    classifDic = {}
    print(movies)
    for movie in movies:
        print(movie)
        classifDic[movie] = get_movie_rating(get_movie_data(movie))
    
    return [i[0] for i in sorted(classifDic.items(), key=lambda item: (item[1], item[0]), reverse=True)]
    
    #return [classifDic[i] for i in index]
    
get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])



In [1]:
# APIrest Itunes 
import requests

param = {"term":"JesusCopy"}

resp = requests.get(" https://itunes.apple.com/search", params=param)

print(resp.text)




{
 "resultCount":3,
 "results": [
{"wrapperType":"audiobook", "artistId":1150439209, "collectionId":1437942418, "artistName":"Douglas Gonçalves", "collectionName":"JesusCopy: A revolução das cópias de Jesus [The Revolution of the Copies of Jesus] (Unabridged)", "collectionCensoredName":"JesusCopy: A revolução das cópias de Jesus [The Revolution of the Copies of Jesus] (Unabridged)", "artistViewUrl":"https://books.apple.com/us/author/douglas-gon%C3%A7alves/id1150439209?uo=4", "collectionViewUrl":"https://books.apple.com/us/audiobook/jesuscopy-a-revolu%C3%A7%C3%A3o-das-c%C3%B3pias-de-jesus/id1437942418?uo=4", "artworkUrl60":"https://is1-ssl.mzstatic.com/image/thumb/Music128/v4/81/74/65/817465be-7efc-a847-3ed8-d1dbba32b180/rm_image.jpg/60x60bb.jpg", "artworkUrl100":"https://is1-ssl.mzstatic.com/image/thumb/Music128/v4/81/74/65/817465be-7efc-a847-3ed8-d1dbba32b180/rm_image.jpg/100x100bb.jpg", "collectionPrice":3.99, "collectionExplicitness":"notExplicit", "trackCount":1, "copyright":"© 

In [2]:
# APIrest flickr
import json

params = {"method":"flickr.photos.search", 
         "api_key":"cf2b7ca39080f1a420965e63392c8ef4", 
         "format":"json", "nojsoncallback":1, 
         "tags":["snow"], 
         "tag_mode":"any", 
         "privacy_filter":1,
         "media":"photos",
         "per_page":2
         }

endpoint = "https://api.flickr.com/services/rest/"

resp = requests.get(endpoint, params=params)

capInfo = json.loads(resp.text)


for p in capInfo["photos"]["photo"]:
    print(f"flickr.com/photos/{p['owner']}/{p['id']}")



flickr.com/photos/47294043@N02/53400699648
flickr.com/photos/66855284@N03/53399569147


## Construíndo Cache com Redis 

In [3]:
### 
import redis

try:
    conn = redis.Redis(host='localhost', port=6379, decode_responses=True)
    conn.ping()
except redis.exceptions.ConnectionError as err:
    print('\n Conn Redis: {0} \n'.format(err))



In [46]:
def seach(keycahe):
    return conn.get(keycahe)

def add(keycache, value):
    conn.set(name=keycache, value=value, ex=900) #15min

In [24]:
def show(resp: json):
    for p in resp["photos"]["photo"]:
        print(f"flickr.com/photos/{p['owner']}/{p['id']}")

In [66]:
params = {"method":"flickr.photos.search", 
         "api_key":"cf2b7ca39080f1a420965e63392c8ef4", 
         "format":"json", "nojsoncallback":1, 
         "tags":["girafa"], 
         "tag_mode":"any", 
         "privacy_filter":1,
         "media":"photos",
         "per_page":2
         }

endpoint = "https://api.flickr.com/services/rest/"

keycache = endpoint+"_".join([str(params[key]) for key in params if key != "api_key"])
#print(keycache)

linkCache = seach(keycache)
#print(resCache)

if linkCache == None:
    try:
        resp = requests.get(endpoint, params=params)
        link = resp.text
        add(keycache, link)
    except requests.exceptions.ConnectionError as err:   # Montar conjunto de ex...
         print('\n requests: {0} \n'.format(err))
else:
    print("cache....")
    link = linkCache

show(json.loads(link))
    



cache....
flickr.com/photos/60083277@N00/53275441083
flickr.com/photos/60083277@N00/53274268012


In [42]:
a = [k for k in params if k != "api_key"]

list(map((lambda x: str(params[x])), [k for k in params if k != "api_key"]))

['flickr.photos.search', 'json', '1', "['snow']", 'any', '1', 'photos', '2']

In [ ]:
# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
import requests_with_caching
import json
# some invocations that we use in the automated tests; uncomment these if you are getting errors and want better error messages
# get_movies_from_tastedive("Bridesmaids")
def get_movies_from_tastedive(name, tp="movies", limit=5):
    urlbase = "https://tastedive.com/api/similar"
    params = dict()
    params["q"] = name 
    params["type"] = tp
    params["limit"] = 5
    resp = requests_with_caching.get(urlbase, params=params)
    date = json.loads(resp.text)
    return date

def extract_movie_titles(date):
    return [name["Name"] for name in date["Similar"]["Results"]]

def get_related_titles(args):
    if len(args) > 0:
        dates = list((map(extract_movie_titles, (map(get_movies_from_tastedive, args)))))
        return list(set(dates[0]+dates[1]))
    else: 
        return []

def get_movie_data(name, tp="json"):
    urlbase = "http://www.omdbapi.com/"
    params = dict()
    params["t"] = name 
    params["r"] = tp
    resp = requests_with_caching.get(urlbase, params=params)
    date = json.loads(resp.text)
    return date

def get_movie_rating(movieInfo):
    if len(movieInfo['Ratings']) > 1:
        num = movieInfo['Ratings'][1]["Value"]
        return int(num.replace("%", ""))
    else:
        return 0
    

def get_sorted_recommendations(args):
    movies = get_related_titles(args)
    classifDic = {}
    print(movies)
    for movie in movies:
        print(movie)
        classifDic[movie] = get_movie_rating(get_movie_data(movie))
    
    return [i[0] for i in sorted(classifDic.items(), key=lambda item: (item[1], item[0]), reverse=True)]
    
    #return [classifDic[i] for i in index]
    
get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])

